## Basic Segmentation Training

In [ ]:
# Change to proper directory
%cd ../MedSegmentation/

/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation


In [ ]:
# Train basic UNet
!python train_Unet.py --train_folder ../pipeline_data_tcia/malign --val_folder ../pipeline_data_tcia/benign --n_epochs 200 --batch_size 1

In [ ]:
# Segment the images of a specific folder
! python api.py --model_dir model_checkpoints --test_images ../pipeline_data_tcia/malign --no_eval

## Combination Segmentation Testing

In [ ]:
# Change to proper directory
%cd ../MedSegmentation/

In [1]:
# Required imports
import sys
sys.path.append("../utils")
from utils import execute_bash_command

In [ ]:
# Combinations to test
# (N Iterations, Batch Size)
seg_combinations = [
    (1000, 1), (1000, 2), (1000, 3),
    (2000, 1), (2000, 2), (2000, 3),
    (3000, 1), (3000, 2), (3000, 3)]

In [2]:
# Performs a single experiment given the parameters
def do_single_experiment(train_folder, val_folder, niters, b_size, test_images):
    
    cmd = f"python train_Unet.py --train_folder {train_folder} --val_folder {val_folder} --n_epochs {niters} --batch_size {b_size}"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
    
    cmd = f"python api.py --model_dir model_checkpoints --test_images {test_images} --no_eval"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
        
    comand = f"zip -r ../(S)NITER{niters}_BSIZE{b_size}.zip ."
    for path in execute_bash_command(comand.split()):
        print(path, end="")
    
    ! rm -r runs results mlruns model_checkpoints model_optimizers

In [ ]:
for comb in seg_combinations:
    do_single_experiment("../pipeline_data_tcia/malign/", "../pipeline_data_tcia/benign/", comb[0], comb[1], "../pipeline_data_tcia/benign/")

## Save Model

In [ ]:
# Zip everything from this model
!zip -r current_model.zip .

## Delete Current Model

In [ ]:
# Delete everything that the model produces
! rm -r runs results mlruns model_checkpoints model_optimizers